#1. 파일 읽기


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec6_머신러닝_무작정따라하기/1. 무조건따라하기/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#2. 데이터 전처리

In [8]:
df_data = df[['Pclass', 'Sex', 'Age']]
df_labels = df['Survived']
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Sex     891 non-null    object 
 2   Age     714 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 21.0+ KB


> 예측할 내용을 객실, 성별, 나이 이 세가지로 구분했다. 현재 나이가 결측치가 존재하는 것으로 나타나며, 성별은 문자열로 처리 되어있다.

##2.2 데이터 변환
> 문자열은 학습을 시킬 수 없음으로 수치데이터로 변환해야 한다.

In [9]:
df_data['Sex'] = df_data['Sex'].map({'female':0, 'male':1})
print(df_data.info())
df_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Sex     891 non-null    int64  
 2   Age     714 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 21.0 KB
None


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Pclass,Sex,Age
0,3,1,22.0
1,1,0,38.0
2,3,0,26.0
3,1,0,35.0
4,3,1,35.0


> 기존에 문자열로 되어있던 성별을 0과 1롤 변환 시켰다.

##2.3 결측치 변환
> 현재 나이에 결측치가 나타나 평균나이로 변환시킬 것이다.

In [10]:
df_data['Age'].fillna(round(df_data['Age'].mean(), 0), inplace=True)
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Sex     891 non-null    int64  
 2   Age     891 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 21.0 KB


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


#3. 모델 만들기

##3.1 정규화

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
standard_scaled = scaler.fit_transform(df_data)

##3.2 데이터 추출

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(standard_scaled, df_labels, test_size=0.3, random_state=62)

##3.3 모델 생성

In [16]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

##3.4 성능 평가

In [17]:
lr.score(X_test, y_test)

0.7947761194029851

##3.5 회귀 계수
> 회귀 계수가 음수가 되면 데이터의 수치가 커질 수록 결과가 작은수일 확률이 높다. 양수가 되면 데이터의 수차가 커질수록 결과가 큰 수일 확률이 높다.

In [18]:
lr.coef_

array([[-0.96121338, -1.20396736, -0.38636416]])

In [19]:
df_data.head()

,Pclass,Sex,Age
0,3,1,22.0
1,1,0,38.0
2,3,0,26.0
3,1,0,35.0
4,3,1,35.0


> 위의 결과에서 Pclass는 -0.96121338을 갖게 되며 음수임으로 데이터가 클수록(객실 등급이 큰 수일수록, 3등급 칸이면) 결과가 작은 값(결과는 0과 1뿐이므로, 0일확률 == 사망했을확률)일 확률이 높다.

> 두 번째 성별 역시 음수임으로 1(남자)일경우 0(죽음)일 확률이 높다.

> 세번째 나이 역시 음수임으로 나이가 많을 수록 죽을 확률이 높다.

In [ ]:
|